In [1]:
# notebook to illustrate document similarity clusters

In [2]:
# following only used for development, reloads the modules with any code changes
%load_ext autoreload
%autoreload 2

# inline matplotlib charts
%matplotlib inline

In [3]:
# import our text mining toolkit
import text_mining_toolkit as tmt

In [8]:
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/simple_test/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/recipes/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(directory_of_files="data_sets/mystery_corpus_01/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/iraq_inquiry/txt/", text_filename_pattern="the-report*.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/clinton_emails/txt/", text_filename_pattern="C0*.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/shakespeare_macbeth/txt/", text_filename_pattern="macbeth_act_0?_scene_0?.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/mixed/txt/", text_filename_pattern="*.txt")
cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/hillsborough/txt/", text_filename_pattern="H*.txt")

content_directory =  data_sets/hillsborough/txt/
text_filename_pattern =  H*.txt
self.documents populated =  4047


In [9]:
# first clear index
tmt.index_wordcount.delete_index(cr.content_directory)

# for all documents in corpus
for document_name in cr.get_documents():
    #print("processing ", document_name)

    # get document text
    document_text = cr.get_text_by_document(document_name)

    # simplify whitespace (remove newlines)
    b = tmt.text_processing.simplify_whitespace(document_text)

    # only keep alphanumeric characters, removes punctuation
    c = tmt.text_processing.keep_only_alphanumeric(b)

    # make lowercase
    d = tmt.text_processing.to_lowercase(c)

    # split into words list
    dl = tmt.text_processing.split_text_into_words(d)
    
    # build n-grams
    #gl = tmt.word_processing.build_ngrams_from_words(dl,2)

    # remove stop words
    el = tmt.word_processing.remove_stop_words(dl, "./stopwords/minimal-stop.txt")

    # only keep words with min length 5
    fl = tmt.word_processing.keep_words_min_length(el,5)
    
    # update index
    tmt.index_wordcount.create_wordcount_index_for_document(cr.content_directory, document_name, fl)
    pass

removed wordcount index file:  data_sets/hillsborough/txt/index_wordcount.hdf5


In [10]:
# merge document indices into a corpus index
tmt.index_wordcount.merge_wordcount_indices_for_corpus(cr.content_directory)

saving corpus word count index ...  data_sets/hillsborough/txt/index_wordcount.hdf5


In [11]:
# delete and build relevance index

tmt.index_relevance.delete_index(cr.content_directory)
tmt.index_relevance.calculate_relevance_index(cr.content_directory)

removed relevance index file:  data_sets/hillsborough/txt/index_relevance.hdf5
saving corpus relevance index ...  data_sets/hillsborough/txt/index_relevance.hdf5


In [ ]:
# calculate document similarities

# delete if already exists
tmt.index_doc_similarity.delete_matrix(cr.content_directory)

# calclate doc1-doc2 similarity
%time tmt.index_doc_similarity.create_doc_similarity_matrix(cr.content_directory)

In [15]:
tmt.index_doc_similarity.print_matrix(cr.content_directory)

doc_similarity_matrix_file  data_sets/simple_test/txt/matrix_docsimilarity.hdf5
          02.txt    03.txt  04.txt  05.txt
01.txt  0.160526  0.095418     0.0     0.0
02.txt  0.000000  0.475525     0.0     0.0
03.txt  0.000000  0.000000     0.0     0.0
04.txt  0.000000  0.000000     0.0     0.0


In [19]:
# get document similarities order by value
%time doc1_doc2_similarity = tmt.index_doc_similarity.get_doc_pairs_by_similarity(cr.content_directory)
doc1_doc2_similarity[:20]

CPU times: user 11.1 ms, sys: 1.39 ms, total: 12.5 ms
Wall time: 13 ms


,doc1,doc2,similarity
0,02.txt,03.txt,1.000000
1,01.txt,02.txt,0.337578
2,01.txt,03.txt,0.200658


In [20]:
# visualise the doc similarities
tmt.visualisation.plot_force_directed_graph(doc1_doc2_similarity[:500])

force-directed graph

In [23]:
# query a word
tmt.index_relevance.search_relevance_index(cr.content_directory, 'orange')

02.txt    0.166731
03.txt    0.111154
01.txt    0.047637
dtype: float64